In [31]:
import rdkit
from rdkit import Chem
import glob
from joblib import Parallel, delayed
import multiprocessing
import argparse, os, gzip
import operator
import numpy as np
from rdkit.Chem import AllChem, rdShapeHelpers
from rdkit.Chem.FeatMaps import FeatMaps
from rdkit import RDConfig

# 3D align all conformations in sdf file (confs path) to reference mol (hit path) and save aligned confs to out_path
def align_and_write(confs_path, hit_path, out_path):
    print('processing ' + confs_path)
    try:
        suppl = Chem.SDMolSupplier(hit_path)
        hit_mol = suppl[0]

        suppl = Chem.SDMolSupplier(confs_path)
        mols = [x for x in suppl]

        for mol in mols:
            o3d = Chem.rdMolAlign.GetO3A(prbMol=mol, refMol=hit_mol)
            o3d.Align()

        writer = Chem.SDWriter(out_path)
        for mol in mols:
            writer.write(mol)
    except: 
        print(str(confs_path + ' FAILED!'))
        
# handle wether to run align_and_write        
def process_align(confs):
    out_path = confs.replace('.sdf', '_aligned.sdf')
    if not os.path.isfile(out_path):
        align_and_write(confs_path=confs, hit_path=hit_file, out_path=out_path)
        
hit_dirs = glob.glob('/Users/res3/michellab/XChem-examples/KALRNA/GDP_frags/GDP_fragweird/')
for f in hit_dirs:
    hit_file = glob.glob(str(f + f.split('/')[-2] + '.sdf'))[0]
    similar_confs_paths = glob.glob(hit_file.replace('.sdf', '-similar-rsn_*_confs.sdf'))
    surprise_confs_paths = glob.glob(hit_file.replace('.sdf', '-surprise-rsn_*_confs.sdf'))
#     print(similar_confs_paths)
    num_cores = multiprocessing.cpu_count()
    Parallel(n_jobs=num_cores)(delayed(process_align)(confs=confs) for confs in similar_confs_paths)
    Parallel(n_jobs=num_cores)(delayed(process_align)(confs=confs) for confs in surprise_confs_paths)



In [33]:
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
m = Chem.MolFromMolFile(hit_file)
m

In [34]:
hit_file


'/Users/res3/michellab/XChem-examples/KALRNA/GDP_frags/GDP_frag3/GDP_frag3.mol'

In [40]:
m = Chem.MolFromMolFile('../GTP.sdf')

In [41]:
if m:
    print('yay')

yay


In [43]:
hit_dirs

['/Users/res3/michellab/XChem-examples/KALRNA/GDP_frags/']

In [17]:
hit_dirs

['/Users/res3/michellab/XChem-examples/KALRNA/GDP_frags/GDP_fragweird/']

In [18]:
hit_file = glob.glob(str(f + f.split('/')[-2] + '.sdf'))

In [22]:
os.path.isfile(f + f.split('/')[-2]+'.sdf')

False

In [34]:
sdfs = glob.glob('GDP_docked/*.sdf')
pdb_mol = Chem.MolFromPDBFile('../XX02KALRNA-x1376_1/XX02KALRNA-x1376_1.pdb')
for sdf in sdfs:
    prb_mols = Chem.SDMolSupplier(sdf, sanitize=True)
    prb_mols = [x for x in prb_mols if x]
    i = 1
    for prb in prb_mols:
        temp_mol = Chem.rdmolops.CombineMols(prb, pdb_mol)
        out_name = sdf.replace('.sdf', str('_' + str(i) + '.pdb'))
        Chem.MolToPDBFile(temp_mol, out_name)
        i +=1
        

In [33]:
temp_mol


In [35]:
import pandas as pd 
import xml.etree.ElementTree as et

In [44]:
x = '/Users/res3/Downloads/report.xml'
xml_string = open(x,'rb').read()

In [45]:
from lxml import objectify

In [46]:
root = objectify.fromstring(xml_string)

In [55]:
len(root["bindingsite"]) #[1]["identifiers"]["hetid"]

2

In [58]:
lig_binding_site = [i for i in root['bindingsite'] 
                   if i["identifiers"]["hetid"]=='UNL']

In [60]:
lig_binding_site

[<Element bindingsite at 0x11d175848>]

In [1]:
import glob
pdbs = glob.glob('/Users/res3/michellab/XChem-examples/KALRNA/GDP_frags/GDP_docked/*.pdb')

In [2]:
from plip.modules.preparation import PDBComplex

In [3]:
plips = []
for pdb in pdbs:
    a = PDBComplex()
    a.load_pdb(pdb)
    a.analyze()
    plips.append(a)

KeyboardInterrupt: 

In [32]:
a = PDBComplex()

In [33]:
a.load_pdb(pdbs[0])

In [4]:
plips[0].interaction_sets['UNL:Z:1']


In [22]:
def get_interaction_set_keys(plip, interaction_search_string):
    keys = []
    for key in plip.interaction_sets.keys():
        if interaction_search_string in key:
            print('found interaction set...')
            print(key)
            keys.append(plip.interaction_sets[key])
    return keys

In [27]:
sets = [plips[0].interaction_sets[key] for key in get_interaction_set(plips[0], 'UNL')]

found interaction set...
UNL:Z:1


In [43]:
sets[0].all_itypes

[hbond(a=<pybel.Atom object at 0x1245f2358>, a_orig_idx=12, d=<pybel.Atom object at 0x124453fd0>, d_orig_idx=1238, h=<pybel.Atom object at 0x1247c10f0>, distance_ah=2.421333616876132, distance_ad=2.9833311918055623, angle=115.70870487695107, type='strong', protisdon=True, resnr=159, restype='ALA', reschain='A', resnr_l=1, restype_l='UNL', reschain_l='Z', sidechain=False, atype='O2', dtype='Nam'),
 hbond(a=<pybel.Atom object at 0x1245f2358>, a_orig_idx=12, d=<pybel.Atom object at 0x124457240>, d_orig_idx=1243, h=<pybel.Atom object at 0x1247c11d0>, distance_ah=2.279257613917704, distance_ad=3.143318787523785, angle=145.89954642839098, type='strong', protisdon=True, resnr=160, restype='LEU', reschain='A', resnr_l=1, restype_l='UNL', reschain_l='Z', sidechain=False, atype='O2', dtype='Nam'),
 hbond(a=<pybel.Atom object at 0x1245711d0>, a_orig_idx=126, d=<pybel.Atom object at 0x12473a9e8>, d_orig_idx=16, h=<pybel.Atom object at 0x124748320>, distance_ah=2.1831958459918446, distance_ad=3.019

In [25]:
print(i_set)

['UNL:Z:1']
